# Workflow:

1. Importing necessary packages
2. Feature exraction and creation of dataset
3. Train - test split
4. Feature scaling
5. Model building
6. Evaluation metrics
7. Random value prediction
8. Saving the model

## Importing necessary packages

In [2]:
import os
import numpy as np
from tensorflow import keras
import librosa
from matplotlib import pyplot
import flask
import pandas as pd
import soundfile as sf
from keras.models import Sequential
from keras.layers import Dense

## Feature extraction using librosa

1. Mel frequency Cepstral coefficients
2. Mel spectrogram
3. Chroma vector
4. Tonal Centroid Features

### Mel frequency cepstral coefficients  

In [12]:
#Defining a function that passses audio file as a parameter to obtain MFCC
def get_mfcc(file_path):
    
    #Converting the mp3 audio data to wav as supported by librosa
    data, samplerate = sf.read(file_path)
    sf.write("temp.wav", data, samplerate)
    
    #Passing the audio file to return MFCC
    y,sr=librosa.load("temp.wav",offset=0,duration=30)
    mfcc=np.array(librosa.feature.mfcc(y=y,sr=sr))
    return mfcc

### Mel Spectrogram 

In [13]:
def get_melspectrogram(file_path):
    
    #Converting the mp3 audio data to wav as supported by librosa
    data, samplerate = sf.read(file_path)
    sf.write("temp.wav", data, samplerate)
    
    #Passing the audio file to return melspectrogram
    y,sr=librosa.load("temp.wav",offset=0,duration=30)
    melspectrogram=np.array(librosa.feature.melspectrogram(y=y,sr=sr))
    return melspectrogram

### Chroma vector 

In [14]:
def get_chroma_vector(file_path):
    #Converting the audio data to wav as supported by librosa
    data, samplerate = sf.read(file_path)
    sf.write("temp.wav", data, samplerate)
    
    #Passing the audio file to return chroma
    y,sr=librosa.load("temp.wav",offset=0,duration=30)
    chroma=np.array(librosa.feature.chroma_stft(y=y,sr=sr))
    return chroma

### Tonal centroid features 

In [15]:
def get_tonnetz(file_path):
    #Converting the audio data to wav as supported by librosa
    data, samplerate = sf.read(file_path)
    sf.write("temp.wav", data, samplerate)
    
    #Passing the audio file to return tonnetz
    y,sr=librosa.load("temp.wav",offset=0,duration=30)
    tonnetz=np.array(librosa.feature.tonnetz(y=y,sr=sr))
    return tonnetz

In [16]:
#Defining a function to extract all features
def get_features(file_path):
    
    #Converting the audio data to wav as supported by librosa
    data, samplerate = sf.read(file_path)
    sf.write("temp.wav", data, samplerate)
    
    #MFCC
    mfcc=get_mfcc("temp.wav")
    mfcc_feature=np.concatenate((mfcc.mean(axis=1),mfcc.min(axis=1),mfcc.max(axis=1)))
    
    #Melspectrogram
    melspectrogram=get_melspectrogram("temp.wav")
    melspectrogram_feature=np.concatenate((melspectrogram.mean(axis=1),melspectrogram.min(axis=1),melspectrogram.max(axis=1)))
    
    #Chroma
    chroma=get_chroma_vector("temp.wav")
    chroma_feature=np.concatenate((chroma.mean(axis=1),chroma.min(axis=1),chroma.max(axis=1)))
    
    #Tonnetz
    tntz=get_tonnetz("temp.wav")
    tntz_feature=np.concatenate((tntz.mean(axis=1),tntz.min(axis=1),tntz.max(axis=1)))
    
    #All features
    feature=np.concatenate((chroma_feature,melspectrogram_feature,mfcc_feature,tntz_feature))
    return feature

## Calculating features for the full dataset 

In [17]:
#Defining directory, genres, feature and labels
directory="/Users/casarulez/Projects/Genre_classification/genres"
genres=["blues","classical","country","disco","hiphop","jazz","metal","pop","reggae","rock"]
features=[]
labels=[]

#Iterating over data directory to calculate features
for genre in genres:
    print("Calculating features for genre:",genre)
    for file in os.listdir(directory+"/"+genre):
            file_path=directory+"/"+genre+"/"+file
            
            features.append(get_features(file_path))
            label=genres.index(genre)
            labels.append(label)
print("Feature calculation complete")

Calculating features for genre: blues
Calculating features for genre: classical
Calculating features for genre: country
Calculating features for genre: disco
Calculating features for genre: hiphop
Calculating features for genre: jazz
Calculating features for genre: metal
Calculating features for genre: pop
Calculating features for genre: reggae
Calculating features for genre: rock
Feature calculation complete


## Splitting the dataset into training, validation and testing 

In [18]:
#Shufling features and labels
permutations=np.random.permutation(1000)
features=np.array(features)[permutations]
labels=np.array(labels)[permutations]

#Training data
features_train=features[0:600]
labels_train=labels[0:600]

#Valutation data
features_val=features[600:800]
labels_val=labels[600:800]

#Testing data
features_test=features[800:1000]
labels_test=labels[800:1000]

#Checking shape of training and testing variables
features_train.shape,features_test.shape,labels_train.shape,labels_test.shape

((600, 498), (200, 498), (600,), (200,))

## Feature scaling 

In [19]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

In [20]:
#Transforming features_train to a standard scale
features_train=sc.fit_transform(features_train)
features_train

array([[-2.21238109, -0.57468604, -2.14202423, ...,  1.53447381,
         1.81224385,  0.92467267],
       [-0.16635519, -0.66663989,  0.13396294, ...,  0.20336491,
         0.09194775,  0.94132004],
       [ 0.50656427,  0.5108785 ,  0.45141395, ..., -0.3082327 ,
        -0.61391297, -0.84090012],
       ...,
       [ 0.34622487,  0.62232654, -0.23390562, ..., -0.81845344,
        -0.78983654, -0.75691484],
       [-0.36225805, -0.70957689, -1.96701227, ...,  0.69206162,
         0.8343396 ,  1.1182848 ],
       [-0.77533632, -0.41126494, -1.08923089, ..., -0.4054274 ,
         0.11173669, -0.26451393]])

In [21]:
#Transforming features_test to a standard scale
features_test=sc.fit_transform(features_test)
features_test

array([[-0.590719  ,  0.10698781,  0.46730602, ..., -0.93119026,
        -0.68793761, -0.57006243],
       [ 2.44140763,  1.87906888,  0.83436794, ..., -0.44275819,
        -0.97444311, -0.58357724],
       [-0.32788661, -0.51633827,  0.74740211, ...,  0.42490856,
        -0.11888527,  0.27133907],
       ...,
       [-0.65551112,  0.52315394, -0.72644979, ...,  0.29331264,
         0.80896654,  0.54202185],
       [-0.47417958,  0.0203973 ,  0.0630657 , ..., -0.26928205,
        -0.64464351, -0.68128041],
       [-0.67014241, -1.37524794, -1.94561459, ..., -0.17864428,
        -0.42300255,  0.55298365]])

## Model building 

In [22]:
#Adding input layer, 2 dense layers and output later
inputs=keras.Input(shape=(498), name="feature")
x=keras.layers.Dense(300,activation="relu",name="dense_1")(inputs)
x=keras.layers.Dense(200,activation="relu",name="dense_2")(x)
outputs=keras.layers.Dense(10,activation="softmax",name="predictions")(x)

#Defining model
model=keras.Model(inputs=inputs,outputs=outputs)

#Compiling model
model.compile(optimizer=keras.optimizers.RMSprop(),loss=keras.losses.SparseCategoricalCrossentropy(),metrics=[keras.metrics.SparseCategoricalAccuracy()])

#Fitting the model
model.fit(features_train.tolist(),labels_train.tolist(),verbose=1,epochs=150)

Epoch 1/150
 1/19 [>.............................] - ETA: 3s - loss: 2.5515 - sparse_categorical_accuracy: 0.0938

2023-10-11 10:31:18.129086: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


19/19 [==============================] - 0s 2ms/step - loss: 1.6024 - sparse_categorical_accuracy: 0.4667
Epoch 2/150
19/19 [==============================] - 0s 2ms/step - loss: 0.7669 - sparse_categorical_accuracy: 0.7783
Epoch 3/150
19/19 [==============================] - 0s 2ms/step - loss: 0.4858 - sparse_categorical_accuracy: 0.8733
Epoch 4/150
19/19 [==============================] - 0s 2ms/step - loss: 0.3197 - sparse_categorical_accuracy: 0.9150
Epoch 5/150
19/19 [==============================] - 0s 2ms/step - loss: 0.2161 - sparse_categorical_accuracy: 0.9400
Epoch 6/150
19/19 [==============================] - 0s 3ms/step - loss: 0.1616 - sparse_categorical_accuracy: 0.9583
Epoch 7/150
19/19 [==============================] - 0s 3ms/step - loss: 0.1021 - sparse_categorical_accuracy: 0.9800
Epoch 8/150
19/19 [==============================] - 0s 3ms/step - loss: 0.0679 - sparse_categorical_accuracy: 0.9850
Epoch 9/150
19/19 [==============================] - 0s 3ms/step - l

19/19 [==============================] - 0s 3ms/step - loss: 3.5429e-05 - sparse_categorical_accuracy: 1.0000
Epoch 69/150
19/19 [==============================] - 0s 3ms/step - loss: 3.4762e-05 - sparse_categorical_accuracy: 1.0000
Epoch 70/150
19/19 [==============================] - 0s 3ms/step - loss: 3.4070e-05 - sparse_categorical_accuracy: 1.0000
Epoch 71/150
19/19 [==============================] - 0s 3ms/step - loss: 3.3423e-05 - sparse_categorical_accuracy: 1.0000
Epoch 72/150
19/19 [==============================] - 0s 3ms/step - loss: 3.2742e-05 - sparse_categorical_accuracy: 1.0000
Epoch 73/150
19/19 [==============================] - 0s 3ms/step - loss: 3.2154e-05 - sparse_categorical_accuracy: 1.0000
Epoch 74/150
19/19 [==============================] - 0s 3ms/step - loss: 3.1564e-05 - sparse_categorical_accuracy: 1.0000
Epoch 75/150
19/19 [==============================] - 0s 3ms/step - loss: 3.1026e-05 - sparse_categorical_accuracy: 1.0000
Epoch 76/150
19/19 [=========

19/19 [==============================] - 0s 3ms/step - loss: 1.4784e-05 - sparse_categorical_accuracy: 1.0000
Epoch 135/150
19/19 [==============================] - 0s 3ms/step - loss: 1.4645e-05 - sparse_categorical_accuracy: 1.0000
Epoch 136/150
19/19 [==============================] - 0s 3ms/step - loss: 1.4518e-05 - sparse_categorical_accuracy: 1.0000
Epoch 137/150
19/19 [==============================] - 0s 3ms/step - loss: 1.4395e-05 - sparse_categorical_accuracy: 1.0000
Epoch 138/150
19/19 [==============================] - 0s 3ms/step - loss: 1.4272e-05 - sparse_categorical_accuracy: 1.0000
Epoch 139/150
19/19 [==============================] - 0s 3ms/step - loss: 1.4137e-05 - sparse_categorical_accuracy: 1.0000
Epoch 140/150
19/19 [==============================] - 0s 3ms/step - loss: 1.4022e-05 - sparse_categorical_accuracy: 1.0000
Epoch 141/150
19/19 [==============================] - 0s 3ms/step - loss: 1.3898e-05 - sparse_categorical_accuracy: 1.0000
Epoch 142/150
19/19 [=

## Model evaluation 

In [23]:
score=model.evaluate(x=features_test.tolist(),y=labels_test.tolist())
print('Accuracy:'+str(score[1]*100)+'%')

7/7 [==============================] - 0s 980us/step - loss: 2.1918 - sparse_categorical_accuracy: 0.6850
Accuracy:68.50000023841858%


## Random value prediction 

In [91]:
def prediction(file_path):
    feature=get_features(file_path)
    feature=feature.reshape(1,498)
    feature=sc.fit_transform(feature)
    y=model.predict(feature)
    ind=numpy.argmax(y)
    print(genres[ind])

In [94]:
prediction("/Users/casarulez/Downloads/in.mp3")

1/1 [==============================] - 0s 10ms/step
rock


## Saving the model 

In [24]:
model.save("genre.h5")